In [ ]:
import sys
sys.path.append("/home/bgreene/LES-utils/")
sys.path.append("/home/bgreene/LES-utils/scripts/")
import xarray as xr
import numpy as np
import seaborn as sb
from LESutils import load_stats
from matplotlib import pyplot as plt
from matplotlib.ticker import ScalarFormatter, NullFormatter, LogFormatterSciNotation, MultipleLocator, LogLocator
from matplotlib import rc, colors
from RFM import recalc_err_array

In [ ]:
# plotting setup
rc("font",weight="normal",size=20,family="serif",serif="Times New Roman")
rc("text",usetex="True")
props=dict(boxstyle="square",facecolor="white",edgecolor="white",alpha=0.0)

# Determine windows for fitting
## Load RFM.nc, var4.nc, stats file

In [ ]:
dnc = "/home/bgreene/simulations/CBL/u01_tw24_qw10_dq-06/"
s = load_stats(dnc+"mean_stats_xyt_5-6h.nc")
RFM_var = xr.load_dataset(dnc+"RFM.nc")
var4 = xr.load_dataset(dnc+"variances_4_order.nc")
L = xr.load_dataset(dnc+"lengthscale.nc")
R = xr.load_dataset(dnc+"autocorr.nc")

## Plot u, w, theta, q

In [ ]:
fig1, ax1 = plt.subplots(nrows=2, ncols=2, sharey=True, figsize=(12, 12))
for jz in np.arange(0, s.nzabl, 4):
    # u
    ax1[0,0].loglog(RFM_var.delta_x, 
                    RFM_var.u_rot.isel(z=jz)/s.u_var_rot.isel(z=jz),
                    label=f"z/zi={(s.z[jz]/s.h).values:4.3f}")
    # w
    ax1[0,1].loglog(RFM_var.delta_x, 
                    RFM_var.w.isel(z=jz)/s.w_var.isel(z=jz),
                    label=f"jz={jz}")
    # theta
    ax1[1,0].loglog(RFM_var.delta_x, 
                    RFM_var.theta.isel(z=jz)/s.theta_var.isel(z=jz))
    # q
    ax1[1,1].loglog(RFM_var.delta_x, 
                    RFM_var.q.isel(z=jz)/s.q_var.isel(z=jz))
    
ax1[0,0].set_ylim([1e-3, 2e0])

# ax1[0,0].set_ylim([1e1, 1e3])
# ax1[0,0].legend(loc="center left")
# ax1[0,1].legend(loc="center left")
# dx_fit_1 = np.array([500, 2000]) #* s.u_mean_rot[64].values
dx_fit_1 = np.array([800, 3000]) #* s.u_mean_rot[64].values
for iax in ax1.flatten():
    iax.axvline(dx_fit_1[0], c="k", ls="--")
    iax.axvline(dx_fit_1[1], c="k", ls="--")

# -1 power law
xx = np.array([1e2, 1e4])
yy = 300 * xx ** -1
# ax1[0,0].loglog(xx, yy, "c")

In [ ]:
plt.loglog(RFM_var.delta_x, RFM_var.u_rot.isel(z=64)/s.u_var_rot[64])
plt.ylim([1e-3, 2e0])

l = Lall[-1]

plt.axvline(s.z[64], c="r")
plt.axvline(10*s.z[64], c="r")
plt.axvline(l.u_rot[64], c="k")
plt.axvline(10*l.u_rot[64], c="k")
# plt.axvline(1800*s.u_mean_rot[64], c="k")
plt.axvline(10)
plt.axvline(100)

# -1 power law
xx = np.array([1e2, 1e4])
yy = 400 * xx ** -1
plt.loglog(xx, yy, "c")

In [ ]:
R.tw.isel(z=1, x=range(2)).integrate("x")

## Plot uw, tw, qw, uu, ww, tt

In [ ]:
fig1, ax1 = plt.subplots(nrows=2, ncols=3, sharey=True, figsize=(18, 12))
for jz in np.arange(0, s.nzabl, 4):
    # uw
    ax1[0,0].loglog(RFM_var.delta_x, 
                    RFM_var.uw.isel(z=jz)/var4.uwuw_var.isel(z=jz),
                    label=f"z/zi={(s.z[jz]/s.h).values:4.3f}")
    # tw
    ax1[0,1].loglog(RFM_var.delta_x, 
                    RFM_var.tw.isel(z=jz)/var4.twtw_var.isel(z=jz),
                    label=f"jz={jz}")
    # qw
    ax1[0,2].loglog(RFM_var.delta_x, 
                    RFM_var.qw.isel(z=jz)/var4.qwqw_var.isel(z=jz))
    # uu
    ax1[1,0].loglog(RFM_var.delta_x, 
                    RFM_var.uu.isel(z=jz)/var4.uuuu_var.isel(z=jz))
    # ww
    ax1[1,1].loglog(RFM_var.delta_x, 
                    RFM_var.ww.isel(z=jz)/var4.wwww_var.isel(z=jz))
    # tt
    ax1[1,2].loglog(RFM_var.delta_x, 
                    RFM_var.tt.isel(z=jz)/var4.tttt_var.isel(z=jz))
    
ax1[0,0].set_ylim([1e-3, 2e0])
# ax1[0,0].legend(loc="center left")
# ax1[0,1].legend(loc="center left")
dx_fit_2 = [300, 1100]
for iax in ax1.flatten():
    iax.axvline(dx_fit_2[0], c="k", ls="--")
    iax.axvline(dx_fit_2[1], c="k", ls="--")

# Load Data

In [ ]:
# directories
# figure save
figdir = "/home/bgreene/CBL-DOE/figures/RFM/"
# list of simulations to loop over
sims = ["u15_tw03_qw01_dq-06", "u15_tw10_qw04_dq-06", "u15_tw24_qw10_dq-06", 
        "u09_tw24_qw10_dq-06"]#, "u01_tw24_qw10_dq-06"]
# empty list to construct paths and store
dnc_all = []
sall = []
Lall = []
Aall = []
err = []
err_LP = []
Call, pall = [], []
# loop over sims and load stats, spectrogram data
for sim in sims:
    dnc = f"/home/bgreene/simulations/CBL/{sim}/"
    dnc_all.append(dnc)
    # load stats file
    s = load_stats(dnc+"mean_stats_xyt_5-6h.nc")
    # add labels using -zi/L
    ziL = -1 * (s.h/s.L).values
    s.attrs["ziL"] = f"$-z_i/L = {ziL:.1f}$"
    sall.append(s)
    # load integral lengthscales
    l = xr.load_dataset(dnc+"lengthscale.nc")
    Lall.append(l)
    # load autocorr files
    a = xr.load_dataset(dnc+"autocorr.nc")
    Aall.append(a)
    # load error files
    e = xr.load_dataset(dnc+"err.nc")
    err.append(e)
    # load error files - LP
    e_LP = xr.load_dataset(dnc+"err_LP.nc")
    err_LP.append(e_LP)
    # load RFM coefficients
    Call.append(xr.load_dataset(dnc+"fit_C.nc"))
    pall.append(xr.load_dataset(dnc+"fit_p.nc"))
    
# define colormap based on number of simulations
nsim = len(sims)
cmap = sb.color_palette("cubehelix_r", nsim)
colors = sb.color_palette("rocket_r", nsim)

In [ ]:
for s in sall:
    plt.plot(np.sqrt(s.u_var_rot)/s.u_mean_rot, s.z/s.h, label=s.ziL)

plt.ylim([0,1])
plt.legend(fontsize=14)
plt.ylabel("$z/z_i$")
plt.xlabel(r"$\sigma_u/U$")

In [ ]:
# plot mean wind speed profiles
fig, ax = plt.subplots(1, 2, sharey=True, sharex=True, figsize=(12,6))
# loop over simulations
for s, e, eLP in zip(sall, err, err_LP):
    ax[0].plot(100*e.wspd, e.z/s.h, label=s.ziL)
    ax[1].plot(100*eLP.wspd, eLP.z/s.h, label=s.ziL)
ax[0].set_ylim([0, 1])
ax[0].set_xlim([0, 200])
ax[0].legend()
ax[0].set_ylabel("$z/h$")
ax[0].set_xlabel(r"$\epsilon_{wspd}^{RFM}$ [\%]")
ax[1].set_xlabel(r"$\epsilon_{wspd}^{LP}$ [\%]")

In [ ]:
# plot mean wind speed profiles
fig, ax = plt.subplots(1, 2, sharey=True, sharex=True, figsize=(12,6))
# loop over simulations
for s, e, eLP in zip(sall, err, err_LP):
    ax[0].plot(100*e.q, e.z/s.h, label=s.ziL)
    ax[1].plot(100*eLP.q, eLP.z/s.h, label=s.ziL)
ax[0].set_ylim([0, 1])
ax[0].set_xlim([0, 100])
ax[0].legend()
ax[0].set_ylabel("$z/h$")
ax[0].set_xlabel("$\\epsilon_{q}^{RFM}$ [%]")
ax[1].set_xlabel("$\\epsilon_{q}^{LP}$ [%]")

In [ ]:
# plot mean wind speed profiles
fig, ax = plt.subplots(1, 2, sharey=True, sharex=True, figsize=(12,6))
# loop over simulations
for s, e, eLP in zip(sall, err, err_LP):
    ax[0].plot(100*e.ww, e.z/s.h, label=s.ziL)
    ax[1].plot(100*eLP.ww, eLP.z/s.h, label=s.ziL)
ax[0].set_ylim([0, 1])
# ax[0].set_xlim([0, 200])
ax[0].legend()
ax[0].set_ylabel("$z/h$")
ax[0].set_xlabel("$\\epsilon_{ww}^{RFM}$ [%]")
ax[1].set_xlabel("$\\epsilon_{ww}^{LP}$ [%]")

# Mean error profiles

In [ ]:
fig, ax = plt.subplots(1, 4, sharey=True, figsize=(14, 4))
for js, (e, s) in enumerate(zip(err, sall)):
    # wspd
    ax[0].plot(100*e.wspd, e.z/s.ht, c=colors[js], lw=3, label=s.ziL)
    # wdir
    ax[1].plot(100*e.wdir, e.z/s.ht, c=colors[js], lw=3)
    # theta
    ax[2].plot(100*e.theta, e.z/s.ht, c=colors[js], lw=3)
    # q
    ax[3].plot(100*e.q, e.z/s.ht, c=colors[js], lw=3, label=s.ziL)

# axis labels
ax[0].set_ylabel("$z/z_i$")
ax[0].set_xlabel(r"$\epsilon_{wspd}$ [\%]")
ax[1].set_xlabel(r"$\epsilon_{wdir}$ [\%]")
ax[2].set_xlabel(r"$\epsilon_{\theta}$ [\%]")
ax[3].set_xlabel(r"$\epsilon_{q}$ [\%]")

# axis limits
ax[0].set_ylim([0, 1])
# ax[0].set_xlim([3e0, 2e4])
# ax[1].set_xlim([1e0, 3e3])
# ax[2].set_xlim([1e-2, 1e1])
# ax[3].set_xlim([5e-1, 2e2])
ax[0].set_xlim([0, 150])
ax[1].set_xlim([0, 34])
ax[2].set_xlim([0, 1.1])
ax[3].set_xlim([0, 75])

# axis ticks and vertical lines
ax[0].yaxis.set_major_locator(MultipleLocator(0.2))
ax[0].yaxis.set_minor_locator(MultipleLocator(0.05))
ax[0].xaxis.set_major_locator(MultipleLocator(50))
ax[0].xaxis.set_minor_locator(MultipleLocator(10))
ax[1].xaxis.set_major_locator(MultipleLocator(10))
ax[1].xaxis.set_minor_locator(MultipleLocator(2))
ax[2].xaxis.set_major_locator(MultipleLocator(0.5))
ax[2].xaxis.set_minor_locator(MultipleLocator(0.1))
ax[3].xaxis.set_major_locator(MultipleLocator(25))
ax[3].xaxis.set_minor_locator(MultipleLocator(5))
# for iax in ax:
#     # iax.xaxis.set_major_formatter(ScalarFormatter())
#     iax.xaxis.set_major_locator(LogLocator(base=10, numticks=15))
#     iax.xaxis.set_minor_locator(LogLocator(base=10, subs=np.linspace(0.1, 0.9, 9),numticks=12))
#     iax.xaxis.set_minor_formatter(NullFormatter())
#     # lines at epsilon = 100%
#     iax.axvline(100, c="k", ls="--", lw=2, label=r"$\epsilon=100\%$")

# legend
ax[3].legend(fontsize=16, bbox_to_anchor=(1, 0.5), frameon=False, loc="center left",
             handlelength=1)

# figure labels
[iax.set_title(f"({l})", loc="left", fontsize=16) for iax, l in zip(ax, list("abcd"))]

In [ ]:
fig, ax = plt.subplots(1, 4, sharey=True, figsize=(14, 4))
for js, (e, s) in enumerate(zip(err, sall)):
    # ustar2
    ax[0].plot(100*e.ustar2, e.z/s.ht, c=colors[js], lw=3, label=s.ziL)
    # t'w'
    ax[1].plot(100*e.tw, e.z/s.ht, c=colors[js], lw=3)
    # q'w'
    ax[2].plot(100*e.qw, e.z/s.ht, c=colors[js], lw=3)
    # tv'tv'
    ax[3].plot(100*e.tvtv, e.z/s.ht, c=colors[js], lw=3, label=s.ziL)

# axis labels
ax[0].set_ylabel("$z/z_i$")
ax[0].set_xlabel(r"$\epsilon_{u_*^2}$ [\%]")
ax[1].set_xlabel(r"$\epsilon_{\overline{\theta'w'}}$ [\%]")
ax[2].set_xlabel(r"$\epsilon_{\overline{q'w'}}$ [\%]")
ax[3].set_xlabel(r"$\epsilon_{\overline{\theta_v'^2}}$ [\%]")

# axis limits
ax[0].set_ylim([0, 1])
ax[0].set_xlim([0, 200])
ax[1].set_xlim([0, 200])
ax[2].set_xlim([0, 150])
ax[3].set_xlim([0, 75])

# axis ticks and vertical lines
ax[0].yaxis.set_major_locator(MultipleLocator(0.2))
ax[0].yaxis.set_minor_locator(MultipleLocator(0.05))
ax[0].xaxis.set_major_locator(MultipleLocator(100))
ax[0].xaxis.set_minor_locator(MultipleLocator(25))
ax[1].xaxis.set_major_locator(MultipleLocator(100))
ax[1].xaxis.set_minor_locator(MultipleLocator(25))
ax[2].xaxis.set_major_locator(MultipleLocator(50))
ax[2].xaxis.set_minor_locator(MultipleLocator(10))
ax[3].xaxis.set_major_locator(MultipleLocator(25))
ax[3].xaxis.set_minor_locator(MultipleLocator(5))

# for iax in ax:
#     # iax.xaxis.set_major_formatter(ScalarFormatter())
#     iax.xaxis.set_major_locator(LogLocator(base=10, numticks=15))
#     iax.xaxis.set_minor_locator(LogLocator(base=10, subs=np.linspace(0.1, 0.9, 9),numticks=12))
#     iax.xaxis.set_minor_formatter(NullFormatter())
#     # lines at epsilon = 100%
#     iax.axvline(100, c="k", ls="--", lw=2, label=r"$\epsilon=100\%$")

# legend
ax[3].legend(fontsize=16, bbox_to_anchor=(1, 0.5), frameon=False, loc="center left",
             handlelength=1)

# figure labels
[iax.set_title(f"({l})", loc="left", fontsize=16) for iax, l in zip(ax, list("abcd"))]

In [ ]:
fig, ax = plt.subplots(1, 4, sharey=True, figsize=(14, 4))
for js, (l, s) in enumerate(zip(Lall, sall)):
    # u
    ax[0].plot(l.u_rot/s.ht, l.z/s.ht, c=colors[js], lw=3, label=s.ziL)
    # w
    ax[1].plot(l.w/s.ht, l.z/s.ht, c=colors[js], lw=3)
    # theta
    ax[2].plot(l.theta/s.ht, l.z/s.ht, c=colors[js], lw=3)
    # q
    ax[3].plot(l.q/s.ht, l.z/s.ht, c=colors[js], lw=3, label=s.ziL)

ax[0].set_ylim([0, 1])
ax[0].set_ylabel("$z/z_i$")
ax[0].yaxis.set_major_locator(MultipleLocator(0.2))
ax[0].yaxis.set_minor_locator(MultipleLocator(0.05))
ax[0].set_xlim([0, 0.7])
ax[0].xaxis.set_major_locator(MultipleLocator(0.3))
ax[0].xaxis.set_minor_locator(MultipleLocator(0.05))
ax[0].set_xlabel(r"$\mathcal{L}_u/z_i$")
ax[1].set_xlim([0, 0.4])
ax[1].xaxis.set_major_locator(MultipleLocator(0.2))
ax[1].xaxis.set_minor_locator(MultipleLocator(0.05))
ax[1].set_xlabel(r"$\mathcal{L}_w/z_i$")
ax[2].set_xlim([0, 0.6])
ax[2].xaxis.set_major_locator(MultipleLocator(0.3))
ax[2].xaxis.set_minor_locator(MultipleLocator(0.05))
ax[2].set_xlabel(r"$\mathcal{L}_\theta/z_i$")
ax[3].set_xlim([0, 0.6])
ax[3].xaxis.set_major_locator(MultipleLocator(0.3))
ax[3].xaxis.set_minor_locator(MultipleLocator(0.05))
ax[3].set_xlabel(r"$\mathcal{L}_q/z_i$")

# legend
ax[3].legend(fontsize=16, bbox_to_anchor=(1, 0.5), frameon=False, loc="center left",
             handlelength=1)

# figure labels
[iax.set_title(f"({l})", loc="left", fontsize=16) for iax, l in zip(ax, list("abcd"))]

## Recalc errors with array of times

In [ ]:
def err_array(dnc, Tnew):
    s = load_stats(dnc+"mean_stats_xyt_5-6h.nc")
    L = xr.load_dataset(dnc+"lengthscale.nc")
    C = xr.load_dataset(dnc+"fit_C.nc")
    p = xr.load_dataset(dnc+"fit_p.nc")
    # add labels using -zi/L
    ziL = -1 * (s.h/s.L).values
    s.attrs["ziL"] = f"$-z_i/L = {ziL:.1f}$"

    return recalc_err_array(Tnew, s, C, p, L), s

In [ ]:
dsim = "/home/bgreene/simulations/CBL/"
sims = ["u15_tw03_qw01_dq-06", "u09_tw24_qw10_dq-06"]
Tnew = np.arange(0.1, 30.1, 0.1)
err_new = []
s_use = []

for sim in sims:
    e, s = err_array(f"{dsim}{sim}/", Tnew)
    err_new.append(e)
    s_use.append(s)

In [ ]:
# PLOT WITH LINEAR COLORBAR
# PLOT WITH LOG COLORBAR
fig, ax = plt.subplots(nrows=2, ncols=4, sharey=True, sharex=True,
                       figsize=(16, 10), constrained_layout=True)

# row 1: -zi/L=1
# row 2: -zi/L=25
cmap_err = sb.color_palette("rocket_r", as_cmap=True)
for js, (s, e) in enumerate(zip(s_use, err_new)):
    # col 0: wspd
    cfax0 = ax[js,0].contourf(Tnew, e.z/s.ht, 100*e.wspd,
                              levels=np.linspace(0, 200, 101), extend="max",
                              cmap=cmap_err)
    # col 1: wdir
    cfax1 = ax[js,1].contourf(Tnew, e.z/s.ht, 100*e.wdir,
                              levels=np.linspace(0, 100, 101), extend="max",
                              cmap=cmap_err)
    # col 2: theta
    cfax2 = ax[js,2].contourf(Tnew, e.z/s.ht, 100*e.theta,
                              levels=np.linspace(0, 1, 101), extend="max",
                              cmap=cmap_err)
    # col 3: q
    cfax3 = ax[js,3].contourf(Tnew, e.z/s.ht, 100*e.q,
                              levels=np.linspace(0, 20, 101), extend="max",
                              cmap=cmap_err)
    if js == 0:
        lvlc = [[20], [5], [0.05], [0.5]]
    else:
        lvlc = [[100], [20], [0.5], [5]]
    # contours for set error level
    ax[js,0].contour(Tnew, e.z/s.ht, 100.*e.wspd, colors="w", levels=lvlc[0], linewidths=4.)
    ax[js,1].contour(Tnew, e.z/s.ht, 100.*e.wdir, colors="w", levels=lvlc[1], linewidths=4.)
    ax[js,2].contour(Tnew, e.z/s.ht, 100.*e.theta, colors="w", levels=lvlc[2], linewidths=4.)
    ax[js,3].contour(Tnew, e.z/s.h, 100.*e.q, colors="w", levels=lvlc[3], linewidths=4.)
    
# colorbars
cb0 = fig.colorbar(cfax0, ax=ax[:,0], location="bottom",
                   ticks=np.linspace(0, 200, 5), pad=0.02,
                   label=r"$\epsilon_{wspd}$ [\%]")
cb1 = fig.colorbar(cfax1, ax=ax[:,1], location="bottom",
                   ticks=np.linspace(0, 100, 5), pad=0.02,
                   label=r"$\epsilon_{wdir}$ [\%]")
cb2 = fig.colorbar(cfax2, ax=ax[:,2], location="bottom",
                   ticks=np.linspace(0, 1, 5), pad=0.02,
                   label=r"$\epsilon_{\theta}$ [\%]")
cb3 = fig.colorbar(cfax3, ax=ax[:,3], location="bottom",
                   ticks=np.linspace(0, 20, 5), pad=0.02,
                   label=r"$\epsilon_{q}$ [\%]")

# axis scales
ax[0,0].set_yscale("log")
# axis limits
ax[0,0].set_ylim([3e-3, 1e0])
ax[0,0].set_xlim([0, 15])
# axis labels
[iax.set_xlabel("Averaging Time [s]") for iax in ax[-1,:]]
[iax.set_ylabel("$z/z_i$") for iax in ax[:,0]]
# axis ticks
ax[0,0].xaxis.set_major_locator(MultipleLocator(3))
ax[0,0].xaxis.set_minor_locator(MultipleLocator(0.5))
# subplot titles
[iax.set_title(f"({lab})", loc="left", fontsize=16) for iax, lab in zip(ax.flatten(), list("abcdefgh"))]
# add zi/L labels on right hand side
ax0b = ax[0,-1].twinx()
ax0b.set_ylabel(s_use[0].ziL)
ax0b.set_yticks([])
ax1b = ax[1,-1].twinx()
ax1b.set_ylabel(s_use[1].ziL)
ax1b.set_yticks([])
# vertical lines for averaging time of typical CopterSonde postprocessing
tav = 5/3.
# legends, vertical lines for each subpanel
# define contour levels from each panel
reflines = [20, 5, 0.05, 0.5, 100, 20, 0.5, 5]
for iax, ref in zip(ax.flatten(), reflines):
    iax.plot([], [], ls="-", c="k", lw=4, label=rf"$\epsilon={ref}\%$")
    iax.axvline(tav, c="w", ls="--", lw=3)
    iax.plot([], [], ls="--", c="k", lw=3, label=f"$T={tav:.1f}$ s")
    # iax.legend(loc="lower right", fontsize=18, frameon=False, handletextpad=0.4,
    #            bbox_to_anchor=(1.05, 0.94), ncols=2, handlelength=1, columnspacing=0.6)
    iax.legend(loc="upper right", handletextpad=0.4, handlelength=1.2, shadow=True)

In [ ]:
# PLOT WITH LOG COLORBAR
fig, ax = plt.subplots(nrows=2, ncols=4, sharey=True, sharex=True,
                       figsize=(16, 10), constrained_layout=True)

# row 1: -zi/L=1
# row 2: -zi/L=1000
cmap_err = sb.color_palette("rocket_r", as_cmap=True)
for js, (s, e) in enumerate(zip(s_use, err_new)):
    # col 0: wspd
    cfax0 = ax[js,0].contourf(Tnew, e.z/s.h, 100*e.wspd,
                              levels=np.logspace(0, 4, 41), extend="max",
                              cmap=cmap_err, locator=LogLocator())
    # col 1: wdir
    cfax1 = ax[js,1].contourf(Tnew, e.z/s.h, 100*e.wdir,
                              levels=np.logspace(0, 4, 41), extend="max",
                              cmap=cmap_err, locator=LogLocator())
    # col 2: theta
    cfax2 = ax[js,2].contourf(Tnew, e.z/s.h, 100*e.theta,
                              levels=np.logspace(-2, 1, 41), extend="max",
                              cmap=cmap_err, locator=LogLocator())
    # col 3: q
    cfax3 = ax[js,3].contourf(Tnew, e.z/s.h, 100*e.q,
                              levels=np.logspace(-1, 2, 41), extend="max",
                              cmap=cmap_err, locator=LogLocator())
    # contours for set error level
    ax[js,0].contour(Tnew, e.z/s.h, 100.*e.wspd, colors="w", levels=[20,5000], linewidths=4.)
    ax[js,1].contour(Tnew, e.z/s.h, 100.*e.wdir, colors="w", levels=[5,1000], linewidths=4.)
    ax[js,2].contour(Tnew, e.z/s.h, 100.*e.theta, colors="w", levels=[0.05,1], linewidths=4.)
    ax[js,3].contour(Tnew, e.z/s.h, 100.*e.q, colors="w", levels=[0.5,20], linewidths=4.)
    
# colorbars
cb0 = fig.colorbar(cfax0, ax=ax[:,0], location="bottom", 
                   ticks=np.logspace(0, 4, 5), pad=0.02,
                   format=LogFormatterSciNotation(10),
                   label=r"$\epsilon_{wspd}$ [\%]")
cb1 = fig.colorbar(cfax1, ax=ax[:,1], location="bottom", 
                   ticks=np.logspace(0, 4, 5), pad=0.02,
                   format=LogFormatterSciNotation(10),
                   label=r"$\epsilon_{wdir}$ [\%]")
cb2 = fig.colorbar(cfax2, ax=ax[:,2], location="bottom", 
                   ticks=np.logspace(-2, 1, 4), pad=0.02,
                   format=LogFormatterSciNotation(10),
                   label=r"$\epsilon_{\theta}$ [\%]")
cb3 = fig.colorbar(cfax3, ax=ax[:,3], location="bottom", 
                   ticks=np.logspace(-1, 2, 4), pad=0.02,
                   format=LogFormatterSciNotation(10),
                   label=r"$\epsilon_{q}$ [\%]")

# axis scales
ax[0,0].set_yscale("log")
# axis limits
ax[0,0].set_ylim([3e-3, 1e0])
ax[0,0].set_xlim([0, 15])
# axis labels
[iax.set_xlabel("Averaging Time [s]") for iax in ax[-1,:]]
[iax.set_ylabel("$z/z_i$") for iax in ax[:,0]]
# axis ticks
ax[0,0].xaxis.set_major_locator(MultipleLocator(3))
ax[0,0].xaxis.set_minor_locator(MultipleLocator(0.5))
# subplot titles
[iax.set_title(f"({lab})", loc="left", fontsize=16) for iax, lab in zip(ax.flatten(), list("abcdefgh"))]
# add zi/L labels on right hand side
ax0b = ax[0,-1].twinx()
ax0b.set_ylabel(s_use[0].ziL)
ax0b.set_yticks([])
ax1b = ax[1,-1].twinx()
ax1b.set_ylabel(s_use[1].ziL)
ax1b.set_yticks([])
# vertical lines for averaging time of typical CopterSonde postprocessing
tav = 5/3.
# legends, vertical lines for each subpanel
# define contour levels from each panel
reflines = [20, 5, 0.05, 0.5, 5000, 1000, 1, 20]
for iax, ref in zip(ax.flatten(), reflines):
    iax.plot([], [], ls="-", c="k", lw=4, label=rf"$\epsilon={ref}\%$")
    iax.axvline(tav, c="w", ls="--", lw=3)
    iax.plot([], [], ls="--", c="k", lw=3, label=f"$T={tav:.1f}$ s")
    # iax.legend(loc="lower right", fontsize=18, frameon=False, handletextpad=0.4,
    #            bbox_to_anchor=(1.05, 0.94), ncols=2, handlelength=1, columnspacing=0.6)
    iax.legend(loc="upper right", handletextpad=0.4, handlelength=1.2, shadow=True)

# Calculate optimal ascent rates and plot

In [ ]:
err_range = [0.50, 0.25, 0.10, 0.05, 0.02, 0.01] # x100 = percent
delta_z = 5. # m
ne = len(err_range)
e = err_new[1]
s = s_use[1]
# determine averaging time to be at/below err for each z
# create empty dataarrays in err for storing
e["t_wspd"] = xr.DataArray(np.zeros((s.nzabl, ne), np.float64),
                             coords=dict(z=e.z, err=err_range))
e["t_wdir"] = xr.DataArray(np.zeros((s.nzabl, ne), np.float64),
                             coords=dict(z=e.z, err=err_range))
e["t_theta"] = xr.DataArray(np.zeros((s.nzabl, ne), np.float64),
                             coords=dict(z=e.z, err=err_range))
e["t_q"] = xr.DataArray(np.zeros((s.nzabl, ne), np.float64),
                             coords=dict(z=e.z, err=err_range))

# loop over error level
for je, lvl in enumerate(err_range):
    # loop over height
    for jz in range(s.nzabl):
        # wspd
        iwspd = np.where(e.wspd.isel(z=jz) <= lvl)[0]
        # check for empty array
        if np.size(iwspd) > 0:
            e["t_wspd"][jz,je] = Tnew[iwspd[0]]
        else:
            e["t_wspd"][jz,je] = np.nan
        # wdir
        iwdir = np.where(e.wdir.isel(z=jz) <= lvl)[0]
        # check for empty array
        if np.size(iwdir) > 0:
            e["t_wdir"][jz,je] = Tnew[iwdir[0]]
        else:
            e["t_wdir"][jz,je] = np.nan
        # theta
        itheta = np.where(e.theta.isel(z=jz) <= lvl)[0]
        # check for empty array
        if np.size(itheta) > 0:
            e["t_theta"][jz,je] = Tnew[itheta[0]]
        else:
            e["t_theta"][jz,je] = np.nan
        # q
        iq = np.where(e.q.isel(z=jz) <= lvl)[0]
        # check for empty array
        if np.size(iq) > 0:
            e["t_q"][jz,je] = Tnew[iq[0]]
        else:
            e["t_q"][jz,je] = np.nan

# now can calculate ascent rate based on fixed delta_z
e["vz_wspd"] = delta_z / e.t_wspd
e["vz_wdir"] = delta_z / e.t_wdir
e["vz_theta"] = delta_z / e.t_theta
e["vz_q"] = delta_z / e.t_q
e.attrs["vz_min"] = s.h / (15 * 60)

In [ ]:
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(12, 4), constrained_layout=True)

ecmap = sb.cubehelix_palette(n_colors=len(err_range), start=0, rot=-1, gamma=1, hue=1, 
                                 light=0.8, dark=0.1, reverse=True)

for je, lvl in enumerate(err_range):
    label = rf"$\epsilon={int(100*lvl)}\%$"
    # wspd
    ax[0].semilogx(e.vz_wspd.isel(err=je), e.z/s.ht, lw=3, c=ecmap[je], label=lvl)
    # wdir
    ax[1].semilogx(e.vz_wdir.isel(err=je), e.z/s.ht, lw=3, c=ecmap[je], label=lvl)
    # q
    ax[2].semilogx(e.vz_q.isel(err=je), e.z/s.ht, lw=3, c=ecmap[je], label=label)

# ax[0].set_ylim([3e-3, 1e0])
ax[0].set_ylim([0, 1])
ax[0].yaxis.set_major_locator(MultipleLocator(0.2))
ax[0].yaxis.set_minor_locator(MultipleLocator(0.05))
ax[0].set_xlim([1e-1, 1e1])
[iax.set_xlabel("Ascent Rate [m s$^{-1}$]")]
ax[0].set_ylabel("$z/z_i$")
ax[0].set_title("Wind Speed")
ax[1].set_title("Wind Direction")
ax[2].set_title("$Q$")

[iax.set_title(f"({l})", loc="left", fontsize=16) for iax, l in zip(ax, list("abc"))]
[iax.set_xlabel("Ascent Rate [m s$^{-1}$]") for iax in ax]
[iax.axvline(3, c="k", lw=2, label=r"$\overline{v}_z$") for iax in ax]
[iax.axvline(e.vz_min, c="#800000", ls="--", lw=2, label="$v_{min}$") for iax in ax]
# legend
ax[2].legend(fontsize=16, bbox_to_anchor=(1, 0.5), frameon=False, loc="center left",
             handlelength=1)
fig.suptitle(s.ziL)